# Introduction to Deep Reinforcement Learning *via* the Arcade Learning Environment

## Introduction
### Background
A quick recap on reinforcement learning and the notations we will adopt here : we address the problem of an **agent** learning to act in an **environment** in order to maximize a **reward** signal.


At each time step, the environment provides an observation $s_t \in S$, the agent responds by selecting an action $a_t \in A$ and then the environment provides a reward $r_{t+1}$ and the next state $s_{t+1}$ with a discount factor $\gamma \in [0, 1]$.
We assume that this system is a Markov Decision Process e.g. the state of the system at the time step $t+1$ only depends on the state and the action chose at the time step $t$.


The goal of RL is to find a policy $\pi$ (e.g. a probability distribution over actions for each state) that maximize the expected discounted return $G_t = \sum_{k=0}^{+\infty}{\gamma^k R_{t+k+1}}$ over an episode in the environment.

To do so, we learn an estimate of the **Q-Value function** $Q_\pi(s_t, a_t) = E_\pi[G_t|s_t, a_t]$ which is equal to the expected reward an agent will receive starting from a state $s_t$ and action $a_t$ and then acting under policy $\pi$. Once this Q-Value function is known, an optimal policy is trivial : $\pi(s_t) = max_{a \in A}{Q(s_t, a)}$.

Let's also define the **Value function** $V_\pi(s_t) = E_\pi[G_t|s_t]$ the expected discounted reward obtained following a policy $\pi$ starting from a state $s_t$.


*For more details, see the [wikipedia page on MDPs][wikiMDP].*



### DQN Algorithm
Theorically, some methods exist to compute the Q-Value function, such as tabular Q-Learning. The idea is to start from a random guess of the Q-Value of each pair (state, action) and iteratively apply [Bellman Equation][BellEq] until it converges toward the real value of the function *(see [this post by Arthur Juliani][JulianiPost] to get an example)*.


But this method is only appliable on very few problems with tiny state and action spaces, and usual problems generally have nearly infinite large state space, and even if the idea is not bad, the tabular approach cannot work on them.


Deep Reinforcement Learning is an efficient method to solve such problems. The idea is to approximate the Q-Value function with a deep neural network trained by gradient descent to then derive a well performing policy from it.
It was successfully implemented for the first time in 2015 [[Mnih et al]][DQN] as DQN by combining the use of convolutional nets to efficiently processes the raw frames fed as input to the network and of a replay memory buffer not to learn only on immediate rewards.


The optimization by gradient descent is realized wrt to the loss on a randomly chosen time step picked from the replay memory $$(R_t + \gamma_{t+1} max_aQ_\bar{\theta}(s_{t+1}, a) - Q\theta(s_t, a_t))^2$$ with $Q_\bar{\theta}$ the *online network* (the network used to select the action) and $Q_\theta$ the *target network* (a copy of the network periodically updated to stabilize the learning).


DQN was the first concrete example of successful reinforcement learning algorithm, and it opened the way to many researches and improvements since 2015. In this notebook, I will present three modified versions of DQN that has lead to improvements or that extended the application domain :
* A3C : an asynchronous version that allows parallelism
* DDPG : a continuous version
* Rainbow DQN : an improved version of DQN, one of the most efficient algorithm of Deep RL today


[wikiMDP]: https://en.wikipedia.org/wiki/Markov_decision_process
[BellEq]: https://en.wikipedia.org/wiki/Bellman_equation#Example
[JulianiPost]: https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0
[DQN]: https://www.nature.com/articles/nature14236

## Asynchronous Advantage Actor-Critic

### Concept
Presented in 2016 [[Mnih et al. again]][A3C], A3C is quite different from regular DQN : it's an Actor-Critic. First, let's debunk the name of the algorithm to understand it better :
+ **Asynchronous :** Asynchronous algorithm is a great family of algos designed to run on parallel architecture to speed the learning up. The main idea is instead of having only one agent interacting with an environment, an asynchronous algo creates a global network and many workers on parallel threads with each their own environment and their own neural network. Then, during an episode, each agent copies the weights of the global network, then interacts with it's own environment and apply a gradient descent on the weight of the global network.

 This has two main advantages :
  - the parallelism of today hardware allows to run tens or even hundreds of workers on different thread, and so to maximize the work done in a given time
  - each worker being independent of the others, they collect a lot of various experience and assure a better exploration of the state space, removing the necessity to have a replay memory buffer


+ **Actor-Critic :** Instead of just estimating the Q-Value function and then inducing a policy by acting greedily with respect to the action Q-Values, in Actor-Critic algorithms, the network estimate both the Value $V$ function and a policy $\pi$. The value estimator is called the **critic** and the policy estimator the **actor**.



+ **Advantage :** We define advantage as the difference between the Q-Value and the Value of a given state and action $A(s, a) = Q(s, a) - V(s)$ : it represents how much better an action is than expected. During a gradient descent, the updates usually use discounted reward to tell the quality of a taken action, but one way to be more efficient is instead to use an advantage estimate to get how much better it is than average.

 We can estimate this advantage quite easily because $V(s_t)$ is the output of our network and $Q(s_t, a_t)$ can be estimated by the discounted reward $R_t = \sum_{k=0}^{+\infty}{\gamma^k r_{t+k}}$ : $$A_{est}(s_t, a_t) = R_t - V(s_t)$$
 
![alt text](./Images/A3C.png "Architecture of an A3C Network")

### Implementation
*This code can be found in ../A3C/Discrete*

Let's now see an actual implementation of the A3C algorithm applied to a simple Atari 2600 game : Pong.
First, the general outline of the code architecture :
- **Network.py** : defines a class Network that contains the neural network definition (convolution + LSTM layers), the Tensorflow operations to compute and apply gradients on the network weights and the operations to copy the weights of the global network to the local one
- **Agent.py** : defines a class Agent that contains a local network and a local environment and who can run episodes to get experience and apply gradient descents to the global network
- **main.py** : the main program that creates a global network and many workers, and run them on different threads
- **settings.py** : contains every important setting that can be modified in the algorithm


- **Environment.py** is a wrapper for [ALE](https://github.com/mgbellemare/Arcade-Learning-Environment) Pong game
- **RMSPropApplier.py** is a modified version of tensorflow optimizer that can be shared between every worker like in the original paper of A3C (code taken from [Miyosuda implentation](https://github.com/miyosuda/async_deep_reinforce))
- **Displayer.py**, **Saver.py**, **eval.py** : defines visualisation, saving and testing tools

Now, let's analyze more precisely the main lines of the code.

[A3C]: https://arxiv.org/pdf/1602.01783.pdf

#### Network

First of all, let's build the neural network 